In [19]:

from sklearn.base import TransformerMixin, BaseEstimator

from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)


from pandas import Series as s , DataFrame as df
import numpy as np

import seaborn as sns
# import matplotlib.pyplot as plt

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc


%matplotlib inline
rc["figure.figsize"] = 10,6

import datetime
import datetime as dt

from sklearn.model_selection  import StratifiedKFold
from sklearn.decomposition import PCA

from iteration_utilities import duplicates, unique_everseen

import sys
from itertools import groupby
from operator import itemgetter
from timeit import timeit


## RandomOverSampler to handle imbalanced data
# from imblearn.over_sampling import RandomOverSampler # over sampling method 2

In [44]:

#Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import MultinomialNB

from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

#preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler, normalize



from sklearn.model_selection import train_test_split

import lightgbm as lgb


#Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support

from sklearn.pipeline import Pipeline

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [241]:
train = pd.read_csv("train_fNxu4vz.csv")

test = pd.read_csv("test_fjtUOL8.csv")
submission = pd.read_csv("sample_submission_HSqiq1Q.csv")

train.shape, test.shape, submission.shape

((164309, 14), (109541, 13), (109541, 2))

In [242]:
df = train.copy() 
df_test = test.copy() 

In [243]:
def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    total_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            total_miss_value_column.append(i)
            print("Type ",dataset[i].dtype," Column-> " , i, ", missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
        
    if higher_miss_value_column:
        print("Higher Missing values in Columns for Delete : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")
    
    return total_miss_value_column, higher_miss_value_column

In [244]:
# get_percentage_miss_value(df)

In [245]:
# get_percentage_miss_value(df_test)

In [246]:
def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            

def check_cloumn_details_type_categorical(dataset):
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            
            print("*"*100)
            print("\n")
            

In [247]:
# check_cloumn_details_type_categorical(df)

In [248]:
# check_cloumn_details_type_float(df)

In [249]:
#Label encoding
def convert_to_numerical_label_encoding(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset

In [250]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    


In [251]:
def majority_imbalanced_dataset(dataset, target_col):
    feature_columns = dataset.columns.tolist()
    feature_columns = [c for c in feature_columns if c not in [target_col]]

    X2_new = dataset[feature_columns]
    Y2_new = dataset[target_col]

    os =  RandomOverSampler(random_state=35)
    X_feature_variables , y_output = os.fit_sample(X2_new, Y2_new)
    
    X_feature_variables[target_col] = y_output
    
    X_feature_variables = X_feature_variables.sample(frac = 1).reset_index(drop = True)
    
    return X_feature_variables

In [252]:
def standardscaler_preprocessing(dataset_train, dataset_test, num_col):
    scaler = StandardScaler()
   
    dataset_train[num_col] = scaler.fit_transform(dataset_train[num_col])

    dataset_test[num_col] = scaler.transform(dataset_test[num_col])
    
    return dataset_train, dataset_test

# <font color ='orange' > Step 1: Rakesh EDA

In [253]:
# get_percentage_miss_value(df)

In [254]:
print(df.shape)

df1 = df.drop(columns = ['Loan_ID', 'Months_Since_Deliquency'])

print(df1.shape)

print(df_test.shape)

df_test1 = df_test.drop(columns = ['Loan_ID', 'Months_Since_Deliquency'])

print(df_test1.shape)

(164309, 14)
(164309, 12)
(109541, 13)
(109541, 11)


In [255]:
df1['Loan_Amount_Requested'] = df1['Loan_Amount_Requested'].str.replace(',','').astype(np.float64)

df_test1['Loan_Amount_Requested'] = df_test1['Loan_Amount_Requested'].str.replace(',','').astype(np.float64)

In [256]:
#Missing value imputation for Train
df1.Annual_Income.fillna(0, inplace = True) # Another option is to impute mean to checck model performance
df1.Home_Owner.fillna('UNKNOWN', inplace = True)
df1.Length_Employed.fillna('UNKNOWN', inplace = True)


#Missing value imputation for test
df_test1.Annual_Income.fillna(0, inplace = True) # Another option is to impute mean to checck model performance
df_test1.Home_Owner.fillna('UNKNOWN', inplace = True)
df_test1.Length_Employed.fillna('UNKNOWN', inplace = True)

In [257]:
#outlier managegment
originalCount = len(df1)
df1 = df1[df1['Annual_Income'] <= 225000]
df1 = df1[df1['Number_Open_Accounts'] <= 30]
df1 = df1[df1['Total_Accounts'] <= 68]

finalCount  = len(df1)
print('Original Count -', originalCount , ', Final Count -', finalCount,  ', Dropped -', originalCount - finalCount, '(', round(finalCount/originalCount,4), '%)') 


Original Count - 164309 , Final Count - 161954 , Dropped - 2355 ( 0.9857 %)


In [258]:
scaling_columns = ["Loan_Amount_Requested", "Debt_To_Income" ,"Inquiries_Last_6Mo" , "Number_Open_Accounts", "Total_Accounts", 
                  ]
label_encoding_columns = ["Length_Employed", "Home_Owner" , "Income_Verified" , "Purpose_Of_Loan"]

In [259]:
df4 = convert_to_numerical_label_encoding(df1.copy())
df_test4 = convert_to_numerical_label_encoding(df_test1.copy())

In [260]:
# df5, df_test5  = standardscaler_preprocessing(df4.copy(), df_test4.copy(), scale_col_2)

In [261]:
X1 = df4.drop("Interest_Rate", axis = 1)

Y1 = df4.loc[:, ["Interest_Rate"]]

In [262]:
# model_xgb = XGBClassifier(max_depth=5, objective='multi:softmax',n_estimators=300, num_classes=3)
# model_xgb.fit(X1, Y1)
# # y_pred_final = model_xgb.predict(df3_test)


In [263]:

%%time
model_lgb= LGBMClassifier(n_estimators= 400, objective='multi:softmax', num_classes=3)
model_lgb.fit(X1,Y1)
# y_pred_final = model_lgb.predict(df2_test)

Wall time: 7.77 s


In [115]:

# model_catg= CatBoostClassifier(depth= 3,l2_leaf_reg= 1e-25,learning_rate= 0.07,
#                                loss_function= 'MultiClass',n_estimators= 200)
# model_catg.fit(X,Y)
# y_pred_final = model_catg.predict(df3_test)

0:	learn: 1.0875212	total: 79.4ms	remaining: 15.8s
1:	learn: 1.0780889	total: 159ms	remaining: 15.7s
2:	learn: 1.0696121	total: 256ms	remaining: 16.8s
3:	learn: 1.0619977	total: 389ms	remaining: 19.1s
4:	learn: 1.0553271	total: 556ms	remaining: 21.7s
5:	learn: 1.0495591	total: 815ms	remaining: 26.4s
6:	learn: 1.0443790	total: 1.07s	remaining: 29.6s
7:	learn: 1.0395901	total: 1.31s	remaining: 31.6s
8:	learn: 1.0352500	total: 1.47s	remaining: 31.2s
9:	learn: 1.0313301	total: 1.61s	remaining: 30.6s
10:	learn: 1.0277080	total: 1.72s	remaining: 29.6s
11:	learn: 1.0244920	total: 1.85s	remaining: 29.1s
12:	learn: 1.0212182	total: 1.97s	remaining: 28.3s
13:	learn: 1.0185376	total: 2.05s	remaining: 27.2s
14:	learn: 1.0158645	total: 2.15s	remaining: 26.5s
15:	learn: 1.0134250	total: 2.27s	remaining: 26.1s
16:	learn: 1.0109586	total: 2.35s	remaining: 25.3s
17:	learn: 1.0087585	total: 2.45s	remaining: 24.8s
18:	learn: 1.0067336	total: 2.53s	remaining: 24.1s
19:	learn: 1.0049122	total: 2.64s	remain

161:	learn: 0.9411189	total: 16s	remaining: 3.76s
162:	learn: 0.9410394	total: 16.1s	remaining: 3.66s
163:	learn: 0.9409106	total: 16.2s	remaining: 3.55s
164:	learn: 0.9407986	total: 16.2s	remaining: 3.45s
165:	learn: 0.9406237	total: 16.3s	remaining: 3.34s
166:	learn: 0.9405445	total: 16.4s	remaining: 3.24s
167:	learn: 0.9403810	total: 16.5s	remaining: 3.13s
168:	learn: 0.9402552	total: 16.5s	remaining: 3.03s
169:	learn: 0.9401526	total: 16.6s	remaining: 2.93s
170:	learn: 0.9400744	total: 16.7s	remaining: 2.83s
171:	learn: 0.9399712	total: 16.7s	remaining: 2.73s
172:	learn: 0.9398818	total: 16.8s	remaining: 2.63s
173:	learn: 0.9397095	total: 16.9s	remaining: 2.52s
174:	learn: 0.9396185	total: 17s	remaining: 2.42s
175:	learn: 0.9395172	total: 17s	remaining: 2.32s
176:	learn: 0.9394023	total: 17.1s	remaining: 2.22s
177:	learn: 0.9392745	total: 17.2s	remaining: 2.13s
178:	learn: 0.9391649	total: 17.3s	remaining: 2.02s
179:	learn: 0.9390911	total: 17.4s	remaining: 1.93s
180:	learn: 0.9389

In [265]:
create_submission_file(model_lgb , "final_lgh_10", df_test4)

'File created successful'

In [67]:
def create_submission_file(model, file_name, test_data):
    
    y_pred_final = model.predict(test_data)

    submission_1 = submission.copy()
    submission_1["Interest_Rate"] = y_pred_final
    submission_1.to_csv(file_name+'.csv', index=False)
    
    return "File created successful"